In [ ]:
%pip install fastf1 --quiet
import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import matplotlib.pyplot as plt

import fastf1.plotting


# Load FastF1's dark color scheme
fastf1.plotting.setup_mpl(mpl_timedelta_support=False, misc_mpl_mods=False,color_scheme='fastf1')
import fastf1

# style.use('dark_background')

In [ ]:
def rotate(xy, *, angle):
    rot_mat = np.array([[np.cos(angle), np.sin(angle)],
                        [-np.sin(angle), np.cos(angle)]])
    return np.matmul(xy, rot_mat)

def plot_track(session, plot_sector=False):
    lap = session.laps.pick_fastest()
    tel = lap.get_telemetry()
    tel["Sector"] = tel['RelativeDistance'].apply( lambda rel_dist: int( np.floor( rel_dist * 3 ) ) )

    # pos = lap.get_pos_data()

    # create new figure and axes
    fig, ax = plt.subplots()

    circuit_info = session.get_circuit_info()
    # Get an array of shape [n, 2] where n is the number of points and the second
    # axis is x and y.

    sector_color=['red', 'deepskyblue', 'yellow']

    if plot_sector:
        for sector in range(3):
            track = tel.loc[ tel["Sector"] == sector , ('X', 'Y')].to_numpy()

            # Convert the rotation angle from degrees to radian.
            track_angle = circuit_info.rotation / 180 * np.pi

            # Rotate and plot the track map.
            rotated_track = rotate(track, angle=track_angle)
            ax.plot(rotated_track[:, 0], rotated_track[:, 1], color=sector_color[sector], markersize=10, linewidth=5)
    else:
        track = tel.loc[:, ('X', 'Y')].to_numpy()

        # Convert the rotation angle from degrees to radian.
        track_angle = circuit_info.rotation / 180 * np.pi

        # Rotate and plot the track map.
        rotated_track = rotate(track, angle=track_angle)
        ax.plot(rotated_track[:, 0], rotated_track[:, 1], color='cyan', markersize=10, linewidth=5)


    offset_vector = [500, 0]  # offset length is chosen arbitrarily to 'look good'

    # Iterate over all corners.
    for _, corner in circuit_info.corners.iterrows():
        # Create a string from corner number and letter
        txt = f"{corner['Number']}{corner['Letter']}"

        # Convert the angle from degrees to radian.
        offset_angle = corner['Angle'] / 180 * np.pi

        # Rotate the offset vector so that it points sideways from the track.
        offset_x, offset_y = rotate(offset_vector, angle=offset_angle)

        # Add the offset to the position of the corner
        text_x = corner['X'] + offset_x
        text_y = corner['Y'] + offset_y

        # Rotate the text position equivalently to the rest of the track map
        text_x, text_y = rotate([text_x, text_y], angle=track_angle)

        # Rotate the center of the corner equivalently to the rest of the track map
        track_x, track_y = rotate([corner['X'], corner['Y']], angle=track_angle)

        # Draw a circle next to the track.
        ax.scatter(text_x, text_y, color='white', s=140)

        # Draw a line from the track to this circle.
        ax.plot([track_x, text_x], [track_y, text_y], color='white')

        # Finally, print the corner number inside the circle.
        ax.text(text_x, text_y, txt, va='center_baseline', ha='center', size='small', color='black')

    ax.set_title(session.event['Location'])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_frame_on(False)
    ax.axis('equal')
    fig.tight_layout()
    # plt.show()
    return fig, ax

In [ ]:
!mkdir -pv tracks

In [ ]:
saved_track = set()

for year in range(2024, 2010, -1):
    try:
        season_schedule = fastf1.get_event_schedule(year, include_testing=False)
    except:
        continue
    for location in season_schedule['Location'].to_list():
        if location in saved_track:
            continue
        try:
            session = fastf1.get_session(year, location, 'Q')
            session.load()
            fig, ax = plot_track(session)
            # fig.show()
            fig.savefig(f"tracks/{location}.png", format='png', dpi=600, bbox_inches='tight')
            saved_track.add(location)
            plt.close(fig)
        except:
            continue

In [ ]:
! zip -r tracks.zip tracks/